In [2]:
%display latex
from pylie import *
from collections.abc import Iterable
import functools
from operator import mul, sub
from pprint import pprint

In [3]:
def reduce(e1: Differential_Polynomial,e2: Differential_Polynomial, context:Context)->Differential_Polynomial:
    # XXX except only function names in context list
    assert e2.is_monic()
    for t, d, c, in zip (e1.terms(), e1.derivatives(), e1.coefficients()):
        ld = e2.Lder()
        def _order (der):
            if der != 1:
                return pylie.order_of_derivative(der)
            else :
                return [0]*len(context._independent)
        e2_order = _order (ld)
        e1_order = _order (d)
        dif = []
        for a, b in zip (e1_order, e2_order):
            dif += [a - b]
        try:
            if ld.operator().function() != d.operator().function():
                continue
        except AttributeError:
            continue            
        if all (map (lambda h: h == 0, dif)) :
            # XXX: do we need that in homogenous environmeent?
            inter = sum(_ for _ in e1.terms()) - c * sum (_ for _ in e2.terms())
            dp = Differential_Polynomial(inter.simplify_full().expand(), context)
            assert dp.is_monic()
            return dp
        elif  all (map (lambda h: h >= 0, dif)):
            print ("PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP")
            # get variables which shuld be differentiate
            variables_to_diff = []
            for i in range (len(context._independent)):
                if dif [i] != 0:
                    variables_to_diff.extend ([context._independent[i]]*abs(dif[i]))
            print ("Vorher", variables_to_diff)
            print ("reduce {} with {}".format(e1.Lder(), e2.Lder()))
            print ("e1:", e1.Lterm())
            print ("e2:", e2.Lterm())
            e1._orig.show()
            e2._orig.show()
            print ("after derivation................")
            diff(e2._orig, *variables_to_diff).show()
            e3 = e1._orig - c * diff(e2._orig, *variables_to_diff)
            e3.show()
            dp =  Differential_Polynomial (e3, context)
            print ("e3:", dp.Lterm())            
            #for _ in dp._p:
            #    print (":3", _._d, _._coeff)            
            return dp
        else:
            print ("no reduction?")
            pass
    return e1

In [4]:
var("x y")
z  = function ("z")(x,y)
ctx = Context((z,), (x,y))
print("*"*70)
e1 = diff(z,y)-x^2/y^2*diff(z,x)-(x-y)*z(x,y)/y^2
e1_dp = Differential_Polynomial (e1, ctx)
print (e1_dp)
print("*"*70)
f1 = diff(z, x) + z(x,y)/x
f1_dp = Differential_Polynomial (f1, ctx)
f1_dp.is_monic()
print (f1_dp)
print("*"*70)
f2 = diff(z, y) + z(x,y)/y
f2_dp = Differential_Polynomial (f2, ctx)
print (f2_dp)

**********************************************************************
-x^2*diff(z(x, y), x)/y^2 - (x - y)*z(x, y)/y^2 + diff(z(x, y), y)
**********************************************************************
z(x, y)/x + diff(z(x, y), x)
**********************************************************************
z(x, y)/y + diff(z(x, y), y)


In [5]:
r1 = reduce (e1_dp, f1_dp, ctx)

In [6]:
r1

In [7]:
r1._orig

-y^2*diff(z(x, y), y)/x^2 - y*z(x, y)/x^2

In [8]:
r1._p

[<pylie.JanetBasis.DTerm object at 0x7fb8cd036d30>,
 <pylie.JanetBasis.DTerm object at 0x7fb8cd036cd0>]

In [9]:
f2_dp._p

[<pylie.JanetBasis.DTerm object at 0x7fb8cb5c5b80>,
 <pylie.JanetBasis.DTerm object at 0x7fb8cb5d41f0>]

In [10]:
r2 = reduce (r1, f2_dp, ctx)

In [11]:
r2

In [12]:
def Lfunc(e):
    return e

In [13]:
def Lcml(e):
    '''Least common left multiply, p.23'''
    return 0

In [14]:
def Lcm(e):
    return 0

In [15]:
def CompleteSystem(S, context):
    return S

In [16]:
def FindIntegrableConditions(S, context):
    return S

In [17]:
def ReduceIntegrabilityConditions(S, context):
    return S

In [31]:
def Reorder (S, context):
    res=sorted(S)
    res.reverse()
    return res

In [19]:
var ("x y")
z = function("z")(x,y)
w = function("w")(x,y)
ctx = Context((w,z), (x,y), MatrixOrder.Mgrlex)
f1 = Differential_Polynomial(diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w(x,y)/y, ctx)
f1._orig

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

In [20]:
f2=Differential_Polynomial(diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x,ctx)
f2._orig

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

In [21]:
f3 = Differential_Polynomial(diff(w, x,y) - 2*x*diff(z, x,2)/y - x*diff(w,x)/y**2 ,ctx)
f3._orig

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

In [22]:
f4 = Differential_Polynomial(diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x* diff(z, y)/y - w(x,y)/(2*y**2), ctx)
f4._orig
f4._p

[<pylie.JanetBasis.DTerm object at 0x7fb8cb5c3280>,
 <pylie.JanetBasis.DTerm object at 0x7fb8cb5c3f40>,
 <pylie.JanetBasis.DTerm object at 0x7fb8c493ed00>,
 <pylie.JanetBasis.DTerm object at 0x7fb8c493ee50>,
 <pylie.JanetBasis.DTerm object at 0x7fb8c493ebb0>,
 <pylie.JanetBasis.DTerm object at 0x7fb8c493e040>]

In [23]:
f5 = Differential_Polynomial(diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w(x,y)/y**2, ctx)
f5._orig

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

In [45]:
def Autoreduce(S, context):  
    i = 0
    while True:
        p = S[i]
        r = S[i+1:]
        if not r:
            S = Reorder(S, context)
            return S[:]            
        have_reduced = False
        checkidx = i+1
        for _r in r:
            print ("Try to reduce r{} with r{}".format(checkidx, i))
            res = reduce (_r, p, context)
            if res._orig != _r._orig:
                print ("have reduced!")
                res._orig.show()
                have_reduced = True                
            S[i] = res
            checkidx += 1
        S = Reorder(S, context)
        print("This is the new System........")
        for _ in S:
            _._orig.show()
        if not have_reduced:
            i += 1        
        else:
            i = 0
            

In [46]:
def JanetBasis(S, context):
    """List of homogenous PDE's + order context"""
    if not isinstance(S, Iterable):
        # bad criterion
        S = [S]
    S = [Differential_Polynomial(s, context) for s in S]
    S = Reorder(S, context)
    while 1:
        S = Autoreduce (S, context)
        #S = CompleteSystem (S, context)
        #f = FindIntegrableConditions(S, context)
        #c = ReduceIntegrabilityConditions(S, context)
        #if all ([_ == 0 for _ in c]) or not c:
        #    return S
        #S += c
        #S = Reorder (S, context)
        return S[:]

In [47]:
checkS=JanetBasis([f3._orig,f1._orig,f5._orig, f2._orig,f4._orig], ctx)

Try to reduce r1 with r0
have reduced!


2*x^2*y*diff(z(x, y), x)/(x^3 + x*y) + 1.0*x^3*diff(z(x, y), x, y)/(x^3 + x*y) + x^2*w(x, y)/(x^3 + x*y) + 2*y^2*diff(z(x, y), x)/(x^3 + x*y) + 1.0*x*y*diff(z(x, y), x, y)/(x^3 + x*y) + y*w(x, y)/(x^3 + x*y) - 1/2*x*diff(z(x, y), y)/(x^3 + x*y)

Try to reduce r2 with r0
PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP
Vorher [y]
reduce diff(w(x, y), y, y) with diff(w(x, y), y)
e1: 1.0*diff(w(x, y), y, y)
e2: 1.0*diff(w(x, y), y)


-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

...

Try to reduce r3 with r1
have reduced!


0

Try to reduce r4 with r1
have reduced!


0

IndexError: list index out of range

In [33]:
for i in (f1._orig, f2._orig,f3._orig, f4._orig,f5._orig):
    i.show()

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

x*diff(z(x, y), y)/y - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), x, y) + diff(z(x, y), x, y)

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

In [34]:
for _ in checkS:
    _._orig.show()

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

-2*x*diff(z(x, y), x, x)/y - 1.0*x*diff(z(x, y), y)/y - x*diff(w(x, y), x)/y^2 + 1.0*diff(w(x, y), x)/y - 0.5*diff(w(x, y), y)/y + 0.5*w(x, y)/y^2 - 1.0*diff(z(x, y), x, y)

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

x*diff(z(x, y), y)/y - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), x, y) + diff(z(x, y), x, y)

In [ ]:
g1 = Differential_Polynomial (diff(z,y,y) + diff(z,y)/(2*y), ctx)
g1._orig

In [ ]:
g2 = Differential_Polynomial(diff(w,x,x) + 4*y*y*diff(w,y) - 8*y*y*diff(z, x) - 8*y*w(x,y), ctx)
g2._orig

In [ ]:
g3 = Differential_Polynomial(diff(w,x,y) - diff(z,x,x)/2 - diff(w,x)/(2*y) - 6*y**2*diff(z,y), ctx)
g3._orig

In [ ]:
g4 = Differential_Polynomial(diff(w,y,y) - 2*diff(z,x,y)- diff(w,y)/(2*y) + w(x,y)/(2*y**2), ctx)
g4._orig

In [ ]:
f1._orig

In [ ]:
f4._orig

In [ ]:
print(f4.Lder(), f1.Lder())

In [ ]:
f4a = reduce(f4, f1, ctx)

In [ ]:
for _ in f4a._p:
   print (_)

In [ ]:
f4a._orig

In [ ]:
wuwu=f4a._p[-1]._coeff.operands()[0]*f4a._p[-1]._coeff.operands()[1].expand().simplify_full()

In [ ]:
f4b = reduce(f4a, f1, ctx)

In [ ]:
f4b._orig

In [ ]:
functools.reduce(operator.mul, f4b._p[1]._coeff.operands()).expand().simplify_full()

In [ ]:
f4c = reduce(f4b, f2, ctx)

In [ ]:
f4c._p[2]._coeff.expand().simplify_full()

In [ ]:
f4d = reduce (f4c, f1, ctx)

In [ ]:
f4d._p[1]._coeff.expand().simplify_full()

In [ ]:
f4d._p[0]._d

In [ ]:
wuwu.collect(y)
# correct according Example 2.35

In [ ]:
wuwu=f4a._p[-2]._coeff

In [ ]:
wuwu

In [ ]:
wuwu=f4a._p[-2]._coeff.operands()

In [ ]:
wuwu

In [ ]:
wawa=wuwu[0]*wuwu[1]

In [ ]:
wawa.expand().simplify_full()

In [ ]:
# wawa is a